In [ ]:
# === AI-Powered Code Auto-Fixer ===
# Uses AST for static analysis + Hugging Face Transformers (free)

import ast
import difflib
from transformers import pipeline

class CodeLinter(ast.NodeVisitor):
    def __init__(self):
        self.unused_imports = []
        self.imports = set()
        self.used_names = set()

    def visit_Import(self, node):
        for alias in node.names:
            self.imports.add(alias.name)
        self.generic_visit(node)

    def visit_ImportFrom(self, node):
        for alias in node.names:
            full_name = f"{node.module}.{alias.name}" if node.module else alias.name
            self.imports.add(full_name)
        self.generic_visit(node)

    def visit_Name(self, node):
        if isinstance(node.ctx, ast.Load):
            self.used_names.add(node.id)
        self.generic_visit(node)

    def lint(self, code):
        tree = ast.parse(code)
        self.visit(tree)
        self.unused_imports = [imp for imp in self.imports if imp.split('.')[-1] not in self.used_names]
        return {
            "unused_imports": self.unused_imports
        }

class CodeAutoFixer:
    def __init__(self):
        # Using Hugging Face CodeBERT-like model via fill-mask task
        self.generator = pipeline("text2text-generation", model="huggingface/CodeBERTa-small-v1")

    def suggest_fixes(self, code, issues):
        prompt = f"Fix the following Python code. Issues: {issues}.\nCode:\n{code}"
        result = self.generator(prompt, max_new_tokens=200)[0]['generated_text']
        return result

    def compare_versions(self, original, fixed):
        diff = difflib.unified_diff(
            original.splitlines(), fixed.splitlines(),
            fromfile='original.py', tofile='fixed.py', lineterm=''
        )
        return '\n'.join(diff)

# Example Usage
if __name__ == "__main__":
    test_code = '''
import os
import sys
import math

def example():
    print(math.pi)
'''
    
    linter = CodeLinter()
    issues = linter.lint(test_code)
    print("Detected Issues:", issues)

    fixer = CodeAutoFixer()
    fixed_code = fixer.suggest_fixes(test_code, issues)

    print("\nSuggested Fix:\n", fixed_code)
    print("\nDifferences:\n")
    print(fixer.compare_versions(test_code, fixed_code))
